In [ ]:
!git clone https://github.com/pytorch/examples.git

In [ ]:
cd examples/word_language_model/

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.decomposition import PCA, KernelPCA
from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import StandardScaler

class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5, tie_weights=False):
        super(RNNModel, self).__init__()
        self.ntoken = ntoken
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError as e:
                raise ValueError( """An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""") from e
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        # Optionally tie weights as in:
        # "Using the Output Embedding to Improve Language Models" (Press & Wolf 2016)
        # https://arxiv.org/abs/1608.05859
        # and
        # "Tying Word Vectors and Word Classifiers: A Loss Framework for Language Modeling" (Inan et al. 2016)
        # https://arxiv.org/abs/1611.01462
        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.encoder.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output)
        decoded = decoded.view(-1, self.ntoken)
        return F.log_softmax(decoded, dim=1), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        else:
            return weight.new_zeros(self.nlayers, bsz, self.nhid)


In [ ]:
# coding: utf-8
import argparse
import time
import math
import os
import torch
import torch.nn as nn
import torch.onnx

import data
# import model

# Set the random seed manually for reproducibility.
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

###############################################################################
# Load data
###############################################################################

corpus = data.Corpus('./data/wikitext-2')

# Starting from sequential data, batchify arranges the dataset into columns.
# For instance, with the alphabet as the sequence and batch size 4, we'd get
# ┌ a g m s ┐
# │ b h n t │
# │ c i o u │
# │ d j p v │
# │ e k q w │
# └ f l r x ┘.
# These columns are treated as independent by the model, which means that the
# dependence of e. g. 'g' on 'f' can not be learned, but allows more efficient
# batch processing.

def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

eval_batch_size = 10

train_data = batchify(corpus.train, eval_batch_size)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)

###############################################################################
# Build the model
###############################################################################

ntokens = len(corpus.dictionary)

nhid = 100
model = RNNModel('LSTM', ntokens, 650, nhid, 2, 0.5).to(device)
# model = model.RNNModel(args.model, ntokens, args.emsize, args.nhid, args.nlayers, args.dropout, args.tied).to(device)

criterion = nn.NLLLoss()

###############################################################################
# Training code
###############################################################################

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""

    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)


# get_batch subdivides the source data into chunks of length args.bptt.
# If source is equal to the example output of the batchify function, with
# a bptt-limit of 2, we'd get the following two Variables for i = 0:
# ┌ a g m s ┐ ┌ b h n t ┐
# └ b h n t ┘ └ c i o u ┘
# Note that despite the name of the function, the subdivison of data is not
# done along the batch dimension (i.e. dimension 1), since that was handled
# by the batchify function. The chunks are along dimension 0, corresponding
# to the seq_len dimension in the LSTM.

def get_batch(source, i):
    seq_len = min(35, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    ntokens = len(corpus.dictionary)

    hidden = model.init_hidden(eval_batch_size)

    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, 35):
            data, targets = get_batch(data_source, i)
            output, hidden = model(data, hidden)
            hidden = repackage_hidden(hidden)
            # output = model(data)
            # output = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output, targets).item()
    return total_loss / (len(data_source) - 1)


def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0.
    start_time = time.time()
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(eval_batch_size)

    for batch, i in enumerate(range(0, train_data.size(0) - 1, 35)):
        data, targets = get_batch(train_data, i)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        model.zero_grad()

        hidden = repackage_hidden(hidden)
        output, hidden = model(data, hidden)
        # output = model(data)
        # output = output.view(-1, ntokens)

        loss = criterion(output, targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        for p in model.parameters():
            p.data.add_(p.grad, alpha=-lr)

        total_loss += loss.item()

        if batch % 200 == 0 and batch > 0:
            cur_loss = total_loss / 200
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // 35, lr,
                elapsed * 1000 / 200, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()
        # if args.dry_run:
        #     break


def export_onnx(path, batch_size, seq_len):
    print('The model is also exported in ONNX format at {}.'.format(os.path.realpath('')))
    model.eval()
    dummy_input = torch.LongTensor(seq_len * batch_size).zero_().view(-1, batch_size).to(device)
    hidden = model.init_hidden(batch_size)
    torch.onnx.export(model, (dummy_input, hidden), path)


# Loop over epochs.
lr = 20
best_val_loss = None

# At any point you can hit Ctrl + C to break out of training early.
try:
    for epoch in range(1, 40+1): # 6+1
        epoch_start_time = time.time()
        train()
        val_loss = evaluate(val_data)
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
                'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                           val_loss, math.exp(val_loss)))
        print('-' * 89)
        # Save the model if the validation loss is the best we've seen so far.
        if not best_val_loss or val_loss < best_val_loss:
            with open('save_model', 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            # Anneal the learning rate if no improvement has been seen in the validation dataset.
            lr /= 4.0
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')



In [ ]:
#### Generate train embedding for training DR approaches
model.eval()
ntokens = len(corpus.dictionary)
hidden = model.init_hidden(eval_batch_size)
total_loss = 0

#### Train or Test
data_source = train_data
train_embed = []

loc = 0

with torch.no_grad():
    for i in range(0, data_source.size(0) - 1, 35):
        data, targets = get_batch(data_source, i)
        emb = model.drop(model.encoder(data))        
        output, hidden = model.rnn(emb, hidden)
        output = model.drop(output)
        train_embed.append(np.array(output.cpu()))

In [ ]:
#### Generate test embedding for testing the reconstructed results from DR approaches
model.eval()
ntokens = len(corpus.dictionary)
hidden = model.init_hidden(eval_batch_size)
total_loss = 0

#### Train or Test
data_source = test_data
test_embed = []

loc = 0

with torch.no_grad():
    for i in range(0, data_source.size(0) - 1, 35): 
        data, targets = get_batch(data_source, i)
        emb = model.drop(model.encoder(data))        
        output, hidden = model.rnn(emb, hidden)
        output = model.drop(output)
        test_embed.append(np.array(output.cpu()))


In [ ]:
train_embed = np.array(train_embed)
test_embed = np.array(test_embed)


In [ ]:
# Preprocessing

X_train = np.zeros((train_embed.shape[0] * train_embed[0].shape[0], train_embed[0].shape[1]*train_embed[0].shape[2]), dtype='float32')
X_test = np.zeros((test_embed.shape[0] * test_embed[0].shape[0], test_embed[0].shape[1]*test_embed[0].shape[2]), dtype='float32')

loc = 0
for i in range(len(train_embed)):
  for j in range(len(train_embed[i])):
    for k in range(len(train_embed[i][j])):
      X_train[loc][k*nhid:(k+1)*nhid] = train_embed[i][j][k]

    loc +=1

X_train = X_train[:loc]

loc = 0
for i in range(len(test_embed)):
  for j in range(len(test_embed[i])):
    for k in range(len(test_embed[i][j])):
      X_test[loc][k*nhid:(k+1)*nhid] = test_embed[i][j][k]

    loc +=1

X_test = X_test[:loc]


print(X_train.shape, X_test.shape)

# Save RAM
del train_embed
del test_embed

In [ ]:
np.save('train_embed.npy', X_train)
np.save('test_embed.npy', X_test)


#Set the target reduced dimension for DR approach

In [ ]:
target_dim = 30 
target_dim *= eval_batch_size # This is for batch size 

#PCA

In [ ]:
# No standardization gives better results.
# We used IncrementalPCA to solve the RAM limitation.
pca = IncrementalPCA(n_components = target_dim)

tr = pca.fit_transform(X_train)

result_test = pca.transform(X_test)

# Inverse
result_test = pca.inverse_transform(result_test)
print(result_test.shape)




#Algo

In [ ]:
#### Algo

train_embed = X_train
test_embed_2 = X_test


# PCA to get Top Components
pca1 =  IncrementalPCA(n_components = train_embed.shape[1])
X_train = train_embed - np.mean(train_embed)
X_fit = pca1.fit_transform(X_train)
U1 = pca1.components_

z = []

# Removing Projections on Top Components
for i, x in enumerate(X_train):
  for u in U1[0:7]:        
          x = x - np.dot(u.transpose(),x) * u 
  z.append(x)

z = np.asarray(z)

# PCA Dim Reduction
pca2 =  IncrementalPCA(n_components = target_dim)
X_train = z - np.mean(z)
X_train_new_final = pca2.fit_transform(X_train)
X_new_final_2 = pca2.transform(test_embed_2)


# PCA to do Post-Processing Again
pca3 =  IncrementalPCA(n_components = target_dim)
X_new = X_train_new_final - np.mean(X_train_new_final)
X_train_new = pca3.fit_transform(X_new)

X_new_2 = pca3.transform(X_new_final_2)

Ufit = pca3.components_

X_new_final_2 = X_new_final_2 - np.mean(X_new_final_2)


for i in range(len(X_new_final_2)):
  for u in Ufit[0:7]:
    X_new_final_2[i] = X_new_final_2[i] - np.dot(u.transpose(), X_new_final_2[i]) * u 

result_test = X_new_final_2

# Inverse
result_test = pca2.inverse_transform(result_test)

print(result_test.shape)

#UMAP

In [ ]:
pip install umap-learn

In [ ]:
import umap
#### UMAP requires a huge RAM and so it is easy to meet Out of Memory.
trans = umap.UMAP(n_neighbors=15, n_components=target_dim, metric='euclidean', low_memory=True).fit(X_train) 

result_test = trans.transform(X_test)
result_test = trans.inverse_transform(result_test)
print(result_test.shape)

#EmbedTextNet

In [ ]:
## Check target_dimension and put it in --target-dim. The value is already multiplied with eval_batch_size.
print(target_dim)

In [ ]:
# Please check !python EmbedTextNet_models.py -h to get information of the argparse in each py file
# Before running this cell, you should upload EmbedTextNet_models.py in appropriate location
!python EmbedTextNet_models.py --train-embed train_embed.npy --test-embed test_embed.npy --target-dim 50 


In [ ]:
test_embed = np.load('EmbedTextNet_test_embed.npy', allow_pickle=True)
print(test_embed.shape)


#Evaluation

In [ ]:
#### You can skip this part for EmbedTextNet since it is already done inside the EmbedTextNet_models.py

test_embed = []

for i in range(0, result_test.shape[0] - 1, 35):
  target = result_test[i:i+35]
  tmp = np.zeros((len(target), eval_batch_size, nhid))
  loc = 0
  for j in range(len(target)):
    tmp[loc] = np.reshape(target[j], [eval_batch_size, nhid])
    loc += 1
  test_embed.append(tmp)

test_embed = np.array(test_embed)

print(test_embed.shape, test_embed[-1].shape)

In [ ]:
model.eval()
ntokens = len(corpus.dictionary)
hidden = model.init_hidden(eval_batch_size)
total_loss = 0

data_source = test_data

loc = 0

with torch.no_grad():
    for i in range(0, data_source.size(0) - 1, 35): # train_data.size(0)
        data, targets = get_batch(data_source, i)

        target = torch.tensor(test_embed[loc]).float().cuda() # cuda()
        decoded = model.decoder(target)
        loc += 1
        decoded = decoded.view(-1, ntokens)

        loss = F.log_softmax(decoded, dim=1)
        total_loss += len(data) * criterion(loss, targets).item()

final_loss = total_loss / (len(data_source) - 1)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    final_loss, math.exp(final_loss)))
print('=' * 89)
